In [0]:
! git clone https://github.com/sigtot/unet-auto unet

Cloning into 'unet'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 449 (delta 248), reused 400 (delta 201), pack-reused 0
Receiving objects: 100% (449/449), 49.59 MiB | 29.75 MiB/s, done.
Resolving deltas: 100% (248/248), done.


In [0]:
! git -C unet checkout master

Already on 'master'
Your branch is up to date with 'origin/master'.


In [0]:
! git -C unet pull

Already up to date.


In [0]:
! git clone https://github.com/sigtot/pix2pix pix2pix

Cloning into 'pix2pix'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 506 (delta 22), reused 32 (delta 11), pack-reused 457
Receiving objects: 100% (506/506), 2.46 MiB | 6.06 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [0]:
! git clone https://github.com/mcordts/cityscapesScripts.git cityscapesScripts


Cloning into 'cityscapesScripts'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 435 (delta 17), reused 22 (delta 8), pack-reused 389
Receiving objects: 100% (435/435), 747.15 KiB | 10.10 MiB/s, done.
Resolving deltas: 100% (211/211), done.


In [0]:
! cd pix2pix && bash ./scripts/eval_cityscapes/download_fcn8s.sh

for details.

--2020-04-17 08:35:22--  http://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/fcn-8s-cityscapes/fcn-8s-cityscapes.caffemodel
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 537897314 (513M)
Saving to: ‘./scripts/eval_cityscapes/caffemodel/fcn-8s-cityscapes.caffemodel’

./scripts/eval_city 100%[===================>] 512.98M   108MB/s    in 4.9s    

2020-04-17 08:35:27 (105 MB/s) - ‘./scripts/eval_cityscapes/caffemodel/fcn-8s-cityscapes.caffemodel’ saved [537897314/537897314]



In [0]:
!apt install -y caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  python3-scipy python3-simplegeneric python3-six python3-skimage
  python3-skimage

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! mkdir -p /content/drive/My\ Drive/citysmall/scripts
! cp -R /content/cityscapesScripts/cityscapesscripts/helpers /content/drive/My\ Drive/citysmall/scripts

In [0]:
%load_ext autoreload
%autoreload 2
import os

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Cityscapes
from torchvision.utils import save_image
from unet.unet import PavelNet
from unet.unet import SigurdModel
import datetime

In [0]:
class CityScapesWithPaths(Cityscapes):
    def __init__(self, root, split='train', mode='fine', target_type='instance',
        transform=None, target_transform=None, transforms=None):
        super(CityScapesWithPaths, self).__init__(root, split=split, mode=mode, target_type=target_type,transform=transform, target_transform=target_transform, transforms=transforms)

    def __getitem__(self, index):
        original_tuple = super(CityScapesWithPaths, self).__getitem__(index)
        path = self.images[index]
        tuple_with_path = ((path,) + original_tuple)
        return tuple_with_path

In [0]:
# Opt for GPU if it's available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")

# Transform to tensor?
transform = transforms.Compose([
    transforms.Resize((286, 286)),
    transforms.CenterCrop((256, 256)),
    transforms.ToTensor(),
])

cityscapes_dir = '/content/drive/My Drive/citysmall' # WARNING: THIS VALUE IS HARDCODED OTHER PLACES
dataset_val = CityScapesWithPaths(cityscapes_dir, transform=transform, target_transform=transform, target_type='color', split="val")
num_imgs = 10  # "all" or int

# From lab
# a function used to transform numpy array to image format
def to_img(x):
    #x = 0.5 * (x + 1)  # This just halves the contrast & makes image brighter?
    x = x.clamp(0, 1)
    x = x.view(x.size(0), x.size(1), x.size(2), x.size(3))
    return x



Using device cuda:0


In [0]:
model = SigurdModel(lr=0.0002, betas=(0.5, 0.999), weight_decay=1e-8, disc_mult=0.5, l1_mult=100, l1_only=False)
model.to(device)

SigurdModel(
  (discriminator): ArtNet(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (3): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (4): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (5): Conv2

In [0]:
model_load_path = "/content/drive/My Drive/models/model_l1.pt"

In [0]:
if os.path.isfile(model_load_path):
    checkpoint = torch.load(model_load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']
    loss_G = checkpoint['loss_G']
    loss_D = checkpoint['loss_D']
    print(f"Loaded {start_epoch + 1} epochs from file")
else:
    print(f"No model found at {model_load_path}")

Loaded 200 epochs from file


In [0]:
# make folders to save output images
pred_dir = '/content/val_img/pred'
orig_dir = '/content/val_img/orig'
mask_dir = '/content/val_img/mask'
os.makedirs(pred_dir, exist_ok=True)
os.makedirs(orig_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

In [0]:
for i, data in enumerate(dataset_val):
    if num_imgs != "all" and i >= num_imgs:
        break
    path, img, mask_with_alpha = data
    fname = path.split("/")[-1]
    save_image(img, './val_img/orig/{}'.format(fname))
    fname_no_split = fname.split("_leftImg8bit")[0]
    save_image(mask_with_alpha, './val_img/mask/{}_gtFine.png'.format(fname_no_split))

    mask = mask_with_alpha[:3, :, :]

  # Add singular fourth dimension to tensor to match input format of model
    mask = mask.unsqueeze(0)
    img = img.unsqueeze(0)

    img = img.to(device)
    mask = mask.to(device)
    
    img_pred = model.forward(mask, img)

    pic = to_img(img_pred.data)
    save_image(img_pred, './val_img/pred/{}'.format(fname))

    if i % 10 == 0:
        print(f"Processed {i}/{len(dataset_val)} images")

Processed 0/500 images


In [0]:
! PYTHONPATH=${PYTHONPATH}:/content/pix2pix/scripts/eval_cityscapes/ bash -c 'cd pix2pix && python ./scripts/eval_cityscapes/evaluate.py --cityscapes_dir "/content/drive/My Drive/citysmall" --result_dir "/content/val_img/pred" --output_dir "/content/fcn_img" --save_output_images 1'

W0412 10:39:20.337648  3984 _caffe.cpp:139] DEPRECATION WARNING - deprecated use of Python interface
W0412 10:39:20.337710  3984 _caffe.cpp:140] Use this instead (with the named "weights" parameter):
W0412 10:39:20.337718  3984 _caffe.cpp:142] Net('./scripts/eval_cityscapes/caffemodel//deploy.prototxt', 1, weights='./scripts/eval_cityscapes/caffemodel/fcn-8s-cityscapes.caffemodel')
I0412 10:39:20.339697  3984 net.cpp:51] Initializing net from parameters: 
state {
  phase: TEST
  level: 0
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 1
      dim: 3
      dim: 500
      dim: 500
    }
  }
}
layer {
  name: "conv1_1"
  type: "Convolution"
  bottom: "data"
  top: "conv1_1"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 64
    pad: 100
    kernel_size: 3
    stride: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type